In [1]:
import pandas as pd
from database.market import Market
from database.sec import SEC
from modeler.modeler import Modeler as m
from preprocessor.model_preprocessor import ModelPreprocessor as mp
from strategy.quarterly_financial_categorical import QuarterlyFinancialCategorical
from datetime import datetime, timedelta, timezone
from backtester.backtester import Backtester as b
import numpy as np
import math
from tqdm import tqdm

In [2]:
market = Market()
sec = SEC("sec")
qfc = QuarterlyFinancialCategorical()

In [3]:
market.connect()
sp5 = market.retrieve_data("sp500")
quarterly_classification_data = market.retrieve_data("dataset_quarter_classification")
quarterly_regression_data = market.retrieve_data("dataset_quarter_regression")
market.close()
classification = quarterly_classification_data[:-1]
regression = quarterly_regression_data[:-1]

In [3]:
categories = qfc.retrieve_data("stock_category_sim")

In [5]:
categories

,_id,year,quarter,ticker,prediction
0,60a7886d3fb2ea95aa81401f,2019,1,ADBE,250Information Technology
1,60a7886d3fb2ea95aa814020,2019,1,ADBE,250Information Technology
2,60a7886d3fb2ea95aa814021,2019,2,ADBE,250Information Technology
3,60a7886d3fb2ea95aa814022,2019,3,ADBE,250Information Technology
4,60a7886d3fb2ea95aa814023,2020,1,ADBE,250Information Technology
...,...,...,...,...,...
33274,60a7a4743fb2ea95aa81c229,2018,4,IR,None
33275,60a7a4743fb2ea95aa81c22a,2018,4,IR,None
33276,60a7a4743fb2ea95aa81c22b,2018,3,IR,None
33277,60a7a4743fb2ea95aa81c22c,2018,4,IR,None


In [6]:
year_gap = 1
training_years = 1
products = []
year_range = range(2012,2021)
quarter_range = range(1,5)
sec.connect()
qfc.db.connect()
for year in tqdm(year_range):
    for quarter in tqdm(quarter_range):
        quarter_categories = categories[(categories["year"] == year) & (categories["quarter"] == quarter)]
        for category in tqdm(quarter_categories["prediction"].unique()):
            group_tickers = quarter_categories[(quarter_categories["prediction"] == category)]
            filings = []
            if group_tickers.index.size > 0:
                for ticker in group_tickers["ticker"].unique():
                    try:
                        ticker = ticker.replace(".","-")
                        cik = sp5[sp5["Symbol"] == ticker]["CIK"].item()
                        filing = sec.retrieve_filing_data(int(cik))
                        drop_columns = ["cik","filed","_id","adsh"]
                        funds = filing.copy()
                        for column in funds.columns:
                            if str(column).islower() and str(column) != "filed":
                                drop_columns.append(column)
                        funds["filed"] = [datetime.strptime(str(x),"%Y%m%d").replace(tzinfo=timezone.utc) if "-" not in str(x) else \
                                            datetime.strptime(str(x).split(" ")[0],"%Y-%m-%d").replace(tzinfo=timezone.utc) for x in funds["filed"]]
                        funds["quarter"] = [x.quarter for x in funds["filed"]]
                        funds["year"] = [x.year + year_gap for x in funds["filed"]]
                        funds["ticker"] = ticker
                        funds.drop(drop_columns,axis=1,inplace=True)
                        qa = funds.copy()
                        for col in qa.columns:
                            test = qa[col].fillna(-99999)
                            availability = 1 - (len([x for x in test if x == -99999]) / qa.index.size)
                            if availability < 0.95:
                                funds.drop(col,inplace=True,axis=1)
                        filings.append(funds)
                    except Exception as e:
                        print(str(e))
                if len (filings) > 0:
                    f = pd.concat(filings)
                    for col in f.columns:
                        test = f[col].fillna(-99999)
                        availability = len([x for x in test != -99999 if x == True]) / test.index.size
                        if availability < 0.7:
                            f.drop(col,axis=1,inplace=True)
                    try:
                        f["y"] = [regression[(regression["year"] == year - year_gap) & (regression["quarter"]==quarter)][row[1]["ticker"]].item() for row in f.iterrows()]
                        f["y_class"] = [classification[(classification["year"] == year-year_gap) & (classification["quarter"]==quarter)][row[1]["ticker"]].item() for row in f.iterrows()]
                        factors = list(f.columns)
                        factors.remove("ticker")
                        for col in f.columns:
                            if col != "ticker":
                                f[col].replace([np.inf, -np.inf,np.nan,np.NaN],f[col].mean(),inplace=True)
                        f = f.groupby(["year","quarter","ticker"]).mean().reset_index()
                        f["date"] = [datetime(row[1]["year"],(row[1]["quarter"] * 3) - 2,1) for row in f.iterrows()]
                        f.dropna(inplace=True)
                        training_data = f[(f["date"] < datetime(year,(quarter*3)-2,1)) & (f["date"]>= datetime(year-year_gap,(quarter*3)-2,1))]
                        prediction_data = f[f["date"]==datetime(year,(quarter*3)-2,1)]
                        X = training_data.drop(["quarter","year","ticker","y_class","y","date"],axis=1)
                        y = training_data[["y_class","y"]]
                        refined_data = {"X":X,"y":y}
                        regression_models = m.regression(refined_data.copy(),ranked=False,tf=False,xgb=True,sk=False,deep=False)
                        classification_models = m.classification(refined_data.copy(),tf=False,xgb=True,sk=False,deep=False,multioutput=False)
                        models = pd.DataFrame([regression_models,classification_models])
                        factors = refined_data["X"].columns
                        product = prediction_data[["year","quarter","ticker"]]
                        for i in range(models.index.size):
                            model = models.iloc[i]["model"]
                            model_type = models.iloc[i]["model_type"]
                            score = models.iloc[i]["score"]
                            predictions = model.predict(prediction_data[factors])
                            product[f"quarterly_financial_categorical_{model_type}_prediction"] = predictions
                            product[f"quarterly_financial_categorical_{model_type}_score"] = score
                        product["year"] = product["year"] + 1
                        qfc.db.store_data("sim",product)
                    except Exception as e:
                        print(str(e))
qfc.db.close()
sec.close()

  0%|                                                                                                                                                                                       | 0/4 [00:00<?, ?it/s]

  0%|                                                                                                                                                                                      | 0/15 [00:00<?, ?it/s]

  7%|███████████▌                                                                                                                                                                  | 1/15 [00:29<06:59, 29.98s/it]

 13%|███████████████████████▏                                                                                                                                                      | 2/15 [00:55<06:11, 28.55s/it]

 27%|██████████████████████████████████████████████▍                                                                                                    

'str' object has no attribute 'predict'




100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [03:41<00:00, 14.79s/it]

 25%|███████████████████████████████████████████▌                                                                                                                                  | 1/4 [03:41<11:05, 221.82s/it]

  0%|                                                                                                                                                                                      | 0/15 [00:00<?, ?it/s]

  7%|███████████▌                                                                                                                                                                  | 1/15 [00:27<06:24, 27.43s/it]

 13%|███████████████████████▏                                                                                                                         

'str' object has no attribute 'predict'




 53%|████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                 | 8/15 [02:22<01:42, 14.71s/it]

 60%|████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                     | 9/15 [02:38<01:31, 15.19s/it]

 67%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                         | 10/15 [03:04<01:31, 18.26s/it]

 73%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                              | 11/15 [03:13<01:02, 15.59s/it]

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍      

...

'str' object has no attribute 'predict'




 93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋            | 13/14 [00:33<00:01,  1.72s/it]

'str' object has no attribute 'predict'




100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:34<00:00,  2.48s/it]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [1:51:58<00:00, 746.55s/it]

'str' object has no attribute 'predict'


In [3]:
qfc.db.connect()
sim = qfc.db.retrieve_data("sim")
qfc.db.close()

In [4]:
sim = sim.dropna().groupby(["year","quarter","ticker"]).mean().reset_index()
sim["quarterly_financial_categorical_classification_prediction"] = [1 if x == 1 else -1 if x == 0 else 0 for x in  sim["quarterly_financial_categorical_classification_prediction"]]

In [5]:
market.connect()
prices = market.retrieve_data("prices").drop("_id",axis=1)
market.close()

In [6]:
sim = prices[["date","year","quarter","ticker","adjclose"]].merge(sim,on=["year","quarter","ticker"],how="left")
sim["quarterly_predicted_delta"]  = (sim["quarterly_financial_categorical_regression_prediction"] - sim["adjclose"]) / sim["adjclose"]
sim = sim.drop("_id",axis=1,errors="ignore")

In [7]:
start = datetime(2013,1,1)
end = datetime(2020,12,23)
extreme = [True,False]
ceiling_settings = [True,False]
fixed = [True,False]
classification = [True,False]
weekly_value_settings = [True,False]
quarterly_delta_range = range(10,40,10)
trade_signal_score_settings = range(60,90,10)
initial = 100

In [8]:
epoch = 0
seats = 5
qfc.db.connect()
qfc.db.drop_table("epochs")
iterations = len(extreme) * len(ceiling_settings) * len(fixed) * len(classification) \
                * len(weekly_value_settings) * len(quarterly_delta_range) \
                 * len(trade_signal_score_settings)
print(iterations)
for i in range(iterations):
    qfc.db.drop_table("trades_{}".format(i))
for e in tqdm(extreme):
    for ceiling in tqdm(ceiling_settings):
        for f in tqdm(fixed):
            for c in classification:
                for weekly_value in weekly_value_settings:
                    for quarterly_delta in quarterly_delta_range:
                        for score in trade_signal_score_settings:
                            s = float(score/100)
                            qd = float(quarterly_delta/100)
                            epoch_dict = {"epoch":epoch
                                          ,"extreme":e
                                          ,"ceiling":ceiling
                                          ,"fixed":f
                                          ,"classification":c
                                          ,"weekly_value":weekly_value
                                          ,"quarterly_delta":qd
                                          ,"score":s
                                         }
                            ts = b.backtest_quarterly(start,end,sim.copy(),seats,e,ceiling,f,c,weekly_value,qd,s)
                            if len(ts) > 0:
                                qfc.db.store_data("epochs",pd.DataFrame([epoch_dict]))
                                qfc.db.store_data("trades_{}".format(epoch),pd.DataFrame(ts))
                            epoch += 1
qfc.db.close()

  0%|                                                                                                                                                                                                                                             | 0/2 [00:00<?, ?it/s]

  0%|                                                                                                                                                                                                                                             | 0/2 [00:00<?, ?it/s]

288




 50%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                                 | 1/2 [44:05<44:05, 2645.78s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [4:04:17<00:00, 7328.66s/it]

 50%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                               | 1/2 [4:04:17<4:04:17, 14657.32s/it]

  0%|                                                                                                                                                                                                   